### 3. Find regions of interest, related topics, type from google trends

In [18]:
import re
import numpy as np
import time

In [19]:
import pandas as pd                        
from pytrends.request import TrendReq
#proxies=['https://34.203.233.13:80',]
pytrends = TrendReq(hl='en-US', tz=530, timeout=(10,25), retries=2, backoff_factor=0.1)

In [20]:
file_name = 'step2_trendy_data.pkl'
df = pd.read_pickle(file_name)

In [22]:
df.head()

,title,popularity
0,#BeKindTowardsAnimals,99.000000
1,#Hyderabad4SSR,73.011033
2,#BiharExitPolls,50.289668
3,#BidenHarris,49.820546
4,#USElectionResults2020,45.314084


#### Finding type

In [23]:
titletype = {}
for topic in df['title']:
    keywords = pytrends.suggestions(keyword=topic)
    #print(keywords)
    try:
        
        if not topic.startswith('hashtag'):
            title = keywords[0]['title']
        else:
            title = topic
        type_of = keywords[0]['type']
    except IndexError:
        title = topic
        type_of = 'hashtag'
    titletype[topic] = type_of
    
    
    

In [25]:
df['Type'] = df['title'].map(titletype)

In [27]:
df.head()

,title,popularity,Type
0,#BeKindTowardsAnimals,99.000000,hashtag
1,#Hyderabad4SSR,73.011033,hashtag
2,#BiharExitPolls,50.289668,hashtag
3,#BidenHarris,49.820546,hashtag
4,#USElectionResults2020,45.314084,hashtag


#### Finding Related Topics

In [28]:
titles = df['title']

related_titles = {}
hashtags = []

for title in titles:
    if title.startswith('#'):
        hasht = title
        hashtags.append(title)
        title = title[1:]
        title = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', title)
        if len(title)<1: 
            continue
        title = ' '.join(title)
        try:
            pytrends.build_payload(kw_list=[title])
            #print(title)
            related_topic = pytrends.related_topics()
            related = list(related_topic[list(related_topic.keys())[0]]['rising']['topic_title'])
            related_titles.update({hasht: related[:4]})
        except Exception as e:
            print(title)
    else:
        try:
            pytrends.build_payload(kw_list=[title])
            #print(title)
            related_topic = pytrends.related_topics()
            related = list(related_topic[list(related_topic.keys())[0]]['rising']['topic_title'])
            related_titles.update({title: related[:4]})
        except Exception as e:
            print(e)
            #print(title,related_topic)
        
        

Ab Hamari Baari Hai


In [29]:
#related_titles

In [30]:
df['Related Topics'] = df['title'].map(related_titles)

In [31]:
df

,title,popularity,Type,Related Topics
0,#BeKindTowardsAnimals,99.000000,hashtag,[Definition]
1,#Hyderabad4SSR,73.011033,hashtag,"[SSR Cricket Ground, Cricket field, Oyo Rooms,..."
2,#BiharExitPolls,50.289668,hashtag,"[Indian general election, 2019, 2016, 2017, Gu..."
3,#BidenHarris,49.820546,hashtag,"[Donald Trump, Vice President of the United St..."
4,#USElectionResults2020,45.314084,hashtag,"[Adult Use of Marijuana Act, Bernie Sanders, M..."
5,Taloja Jail,40.475046,Topic,"[Navi Mumbai, Distance, Mumbai Central Prison ..."
6,#AbHamariBaariHai,40.031399,hashtag,NaN
7,NEET results,36.106092,Entrance examination,"[2017, NEET · 2017, NEET · 2019, NEET · 2018]"
8,India-US,34.407401,Airline,"[Donald Trump, Logistics, Bharatiya Janata Par..."
9,Kamala Harris,28.413920,United States Senator,"[Donald Trump, Vice President of the United St..."


#### Finding Region of Interest

In [32]:
regions_of_interest = {}

for title in titles:
    hasht = title
    if title.startswith('#'):
        hasht = title
        title = title[1:]
        title = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', title)
        if len(title)<1: 
            continue
        title = ' '.join(title)
        
    
    kw_list = [title]
    pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='IN', gprop='')
    regions = pytrends.interest_by_region(resolution='DMA', inc_low_vol=True, inc_geo_code=False)
    regions['region'] = regions.index
    regions = regions.to_numpy()
    regions = sorted(regions, key=lambda regions: regions[0]) [::-1]
    
    regions_of_interest.update({hasht:regions})

        
    print(title)
    time.sleep(5)


Be Kind Towards Animals
Hyderabad SSR
Bihar Exit Polls
Biden Harris
US Election Results
Taloja Jail
Ab Hamari Baari Hai
NEET results
India-US
Kamala Harris
LK Advani
US President
Suresh Raina
America
Samsung
WBB
AEW Full Gear
Arnab Goswamy
Black Day
PM Kisan Yojana
Lok Sabha Elections
President
Israel
Kabir Singh
Avengers: Endgame
Cricket World Cup
Save Arnab
Howdy Modi
De Monetisation
Article 370
Joker
Chandrayaan 2
Captain Marvel
White House
Doland
Bye Bye Trump
Sunday Thoughts
Connecting India


In [33]:
df['regions_of_interest'] = df['title'].map(regions_of_interest)

In [34]:
df

,title,popularity,Type,Related Topics,regions_of_interest
0,#BeKindTowardsAnimals,99.000000,hashtag,[Definition],"[[0, West Bengal], [0, Uttarakhand], [0, Uttar..."
1,#Hyderabad4SSR,73.011033,hashtag,"[SSR Cricket Ground, Cricket field, Oyo Rooms,...","[[100, Telangana], [0, West Bengal], [0, Uttar..."
2,#BiharExitPolls,50.289668,hashtag,"[Indian general election, 2019, 2016, 2017, Gu...","[[100, Goa], [68, Jammu and Kashmir], [28, Ass..."
3,#BidenHarris,49.820546,hashtag,"[Donald Trump, Vice President of the United St...","[[100, Mizoram], [87, Sikkim], [62, Arunachal ..."
4,#USElectionResults2020,45.314084,hashtag,"[Adult Use of Marijuana Act, Bernie Sanders, M...","[[100, Mizoram], [70, Karnataka], [68, Telanga..."
5,Taloja Jail,40.475046,Topic,"[Navi Mumbai, Distance, Mumbai Central Prison ...","[[100, Maharashtra], [12, Uttarakhand], [9, Jh..."
6,#AbHamariBaariHai,40.031399,hashtag,NaN,"[[0, West Bengal], [0, Uttarakhand], [0, Uttar..."
7,NEET results,36.106092,Entrance examination,"[2017, NEET · 2017, NEET · 2019, NEET · 2018]","[[100, Andhra Pradesh], [66, Jammu and Kashmir..."
8,India-US,34.407401,Airline,"[Donald Trump, Logistics, Bharatiya Janata Par...","[[100, Delhi], [47, Karnataka], [41, Maharasht..."
9,Kamala Harris,28.413920,United States Senator,"[Donald Trump, Vice President of the United St...","[[100, Arunachal Pradesh], [86, Mizoram], [83,..."


In [35]:
df.to_pickle('step3_data.pkl')